In [1]:
import pandas as pd
import pymssql
import requests
import uuid
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Azure SQL Server Configuration
SERVER = 'almagraby.database.windows.net'
DATABASE = 'Olist_dataset'
USERNAME = 'DEPI_Projecte'
PASSWORD = 'Almaghraby@240'
TABLE_NAME = 'olist_geolocation_dataset_olist_geolocation_dataset'  # Table containing geolocation_city column

# Azure Translator Configuration
AZURE_TRANSLATOR_KEY = 'AunHlUi5xLkxOBfcQnwIYmiLxh2PztKTD5b5Kbfu4uiLHt1alQKdJQQJ99BJACF24PCXJ3w3AAAbACOGZEWL'  # From Azure Portal
AZURE_TRANSLATOR_REGION = 'uaenorth'  # Your translator region (e.g., eastus, westeurope)
AZURE_TRANSLATOR_ENDPOINT = 'https://api.cognitive.microsofttranslator.com'

In [3]:
# Performance Settings
BATCH_SIZE = 100
MAX_WORKERS = 5
DB_UPDATE_BATCH_SIZE = 50  # CRITICAL: Smaller batches for UPDATE statements

def get_connection():
    """Create a new database connection with retry logic"""
    max_retries = 3
    retry_delay = 5
    
    for attempt in range(max_retries):
        try:
            conn = pymssql.connect(
                server=SERVER,
                user=USERNAME,
                password=PASSWORD,
                database=DATABASE,
                port=1433,
                tds_version='7.4',
                timeout=60,
                login_timeout=60
            )
            return conn
        except pymssql.OperationalError as e:
            error_code = e.args[0] if e.args else None
            
            if error_code == 40615:
                # Firewall blocking
                print(f"\n⚠️  FIREWALL ERROR (Attempt {attempt + 1}/{max_retries})")
                print("=" * 70)
                print("Your Azure SQL Server firewall is blocking this connection.")
                print("\nTO FIX THIS:")
                print("1. Get your current IP:")
                
                try:
                    import requests
                    current_ip = requests.get('https://api.ipify.org', timeout=5).text
                    print(f"   Your IP: {current_ip}")
                except:
                    print("   Run: import requests; print(requests.get('https://api.ipify.org').text)")
                
                print("\n2. Add to Azure Firewall:")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Click '+ Add firewall rule'")
                print(f"   • Start IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print(f"   • End IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print("   • Click 'Save' and wait 2-5 minutes")
                print("\n3. OR Enable 'Allow Azure services' (Recommended):")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Toggle ON: 'Allow Azure services and resources to access this server'")
                print("   • Click 'Save'")
                print("=" * 70)
                
                if attempt < max_retries - 1:
                    print(f"\nWaiting {retry_delay} seconds before retry...")
                    time.sleep(retry_delay)
                else:
                    print("\n❌ Max retries reached. Please fix firewall and re-run.")
                    raise
            else:
                # Other connection errors
                print(f"\n❌ Connection Error: {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    raise
        except Exception as e:
            print(f"\n❌ Unexpected Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                raise
    
    raise Exception("Failed to connect after all retries")

def translate_batch_azure(cities):
    """Translate a batch of cities using Azure Translator API"""
    if not cities:
        return {}
    
    valid_cities = [city for city in cities if pd.notna(city) and city != '']
    
    if not valid_cities:
        return {city: city for city in cities}
    
    path = '/translate'
    constructed_url = AZURE_TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'auto',
        'to': 'en'
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    body = [{'text': str(city)} for city in valid_cities]
    translations = {}
    
    try:
        response = requests.post(constructed_url, params=params, headers=headers, json=body)
        response.raise_for_status()
        results = response.json()
        
        for city, result in zip(valid_cities, results):
            translated_text = result['translations'][0]['text']
            translations[city] = translated_text
        
        for city in cities:
            if pd.isna(city) or city == '':
                translations[city] = city
        
    except Exception as e:
        print(f"Translation error: {e}")
        translations = {city: city for city in cities}
    
    return translations

def translate_all_cities(unique_cities, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    """Translate all unique cities using Azure Translator with parallel processing"""
    print(f"Translating {len(unique_cities)} unique cities...")
    
    city_batches = [unique_cities[i:i + batch_size] 
                    for i in range(0, len(unique_cities), batch_size)]
    
    all_translations = {}
    completed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {executor.submit(translate_batch_azure, batch): batch 
                          for batch in city_batches}
        
        for future in as_completed(future_to_batch):
            try:
                translations = future.result()
                all_translations.update(translations)
                completed += len(translations)
                progress = (completed * 100) // len(unique_cities)
                print(f"Progress: {completed}/{len(unique_cities)} cities ({progress}%)")
            except Exception as e:
                print(f"Batch processing error: {e}")
    
    return all_translations

def update_database_small_batches(translations, table_name, batch_size=DB_UPDATE_BATCH_SIZE):
    """
    Update database in SMALL batches to avoid query size limits
    Uses individual UPDATE statements, not CASE
    """
    if not translations:
        return
    
    valid_translations = {k: v for k, v in translations.items() 
                         if k and v and pd.notna(k) and pd.notna(v)}
    
    if not valid_translations:
        print("No valid translations to update")
        return
    
    total = len(valid_translations)
    print(f"Updating database with {total} translations in batches of {batch_size}...")
    
    # Convert to list for batching
    items = list(valid_translations.items())
    updated_count = 0
    conn = None
    
    try:
        for i in range(0, len(items), batch_size):
            batch = items[i:i + batch_size]
            
            # Create new connection for each batch
            if conn:
                try:
                    conn.close()
                except:
                    pass
            
            conn = get_connection()
            cursor = conn.cursor()
            
            # Update each item individually within transaction
            for original, translated in batch:
                try:
                    update_query = """
                    UPDATE {} 
                    SET geolocation_city_en = %s 
                    WHERE geolocation_city = %s 
                    AND (geolocation_city_en IS NULL OR geolocation_city_en = '')
                    """.format(table_name)
                    
                    cursor.execute(update_query, (translated, original))
                    updated_count += 1
                    
                except Exception as e:
                    print(f"Error updating '{original}': {e}")
                    continue
            
            # Commit batch
            conn.commit()
            cursor.close()
            
            # Progress update
            progress = ((i + len(batch)) * 100) // total
            print(f"Database update progress: {updated_count}/{total} ({progress}%)")
    
    except Exception as e:
        print(f"Database update error: {e}")
    finally:
        if conn:
            try:
                conn.close()
            except:
                pass
    
    print(f"✓ Successfully updated {updated_count} records")

def main():
    try:
        start_time = time.time()
        
        print("=" * 70)
        print("Azure SQL Geolocation Translation Tool - Azure Translator API")
        print("=" * 70)
        
        # Check configuration
        if AZURE_TRANSLATOR_KEY == 'your_translator_key_here':
            print("❌ ERROR: Please update AZURE_TRANSLATOR_KEY")
            print("\nGet your key from:")
            print("Azure Portal → Translator Resource → Keys and Endpoint → Key 1")
            return
        
        # Display current IP for troubleshooting
        print("\n📍 Getting your current IP address...")
        try:
            import requests
            current_ip = requests.get('https://api.ipify.org', timeout=5).text
            print(f"✓ Your current IP: {current_ip}")
            print("  (Use this IP if you need to add a firewall rule)")
        except:
            print("  (Could not detect IP)")
        
        print("\n🔌 Connecting to Azure SQL Server...")
        print(f"   Server: {SERVER}")
        print(f"   Database: {DATABASE}")
        
        conn = get_connection()
        cursor = conn.cursor()
        print("✓ Connected successfully!")
        
        # Check/create translation column
        print("\n📋 Checking table structure...")
        check_column_query = f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = '{TABLE_NAME}' 
            AND COLUMN_NAME = 'geolocation_city_en'
        )
        BEGIN
            ALTER TABLE {TABLE_NAME}
            ADD geolocation_city_en NVARCHAR(255)
        END
        """
        cursor.execute(check_column_query)
        conn.commit()
        print("✓ Table structure verified")
        
        # Read untranslated unique cities
        print(f"\n📖 Reading untranslated cities from {TABLE_NAME}...")
        query = f"""
        SELECT DISTINCT geolocation_city 
        FROM {TABLE_NAME} 
        WHERE (geolocation_city_en IS NULL OR geolocation_city_en = '')
        AND geolocation_city IS NOT NULL
        AND geolocation_city != ''
        """
        df = pd.read_sql(query, conn)
        
        cursor.close()
        conn.close()
        
        total_unique = len(df)
        print(f"✓ Found {total_unique:,} unique cities to translate")
        
        if total_unique == 0:
            print("\n✅ All cities are already translated!")
            return
        
        # Get list of unique cities
        unique_cities = df['geolocation_city'].tolist()
        
        # Estimate characters
        total_chars = sum(len(str(city)) for city in unique_cities)
        print(f"\n📊 Estimated characters: {total_chars:,}")
        print(f"   Azure Free Tier: 2,000,000/month")
        print(f"   Percentage: {(total_chars/2000000)*100:.2f}%")
        
        if total_chars > 2000000:
            print("   ⚠️  WARNING: May exceed free tier limit")
        
        # Translate all unique cities
        print(f"\n🌐 Starting translation...")
        print(f"   Batch size: {BATCH_SIZE} cities/request")
        print(f"   Parallel workers: {MAX_WORKERS}")
        
        translations = translate_all_cities(unique_cities, BATCH_SIZE, MAX_WORKERS)
        
        # Update database in small batches
        print(f"\n💾 Updating database...")
        print(f"   Update batch size: {DB_UPDATE_BATCH_SIZE} cities")
        
        update_database_small_batches(translations, TABLE_NAME, DB_UPDATE_BATCH_SIZE)
        
        # Show sample results
        print("\n📋 Sample translations:")
        conn = get_connection()
        sample_query = f"""
        SELECT TOP 10 geolocation_city, geolocation_city_en 
        FROM {TABLE_NAME} 
        WHERE geolocation_city_en IS NOT NULL
        ORDER BY geolocation_city
        """
        sample_df = pd.read_sql(sample_query, conn)
        print(sample_df.to_string(index=False))
        conn.close()
        
        # Show statistics
        elapsed_time = time.time() - start_time
        print(f"\n" + "=" * 70)
        print(f"✅ TRANSLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print(f"📊 Statistics:")
        print(f"   Unique cities translated: {total_unique:,}")
        print(f"   Characters processed: {total_chars:,}")
        print(f"   Time taken: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
        if elapsed_time > 0:
            print(f"   Average speed: {total_unique/elapsed_time:.2f} cities/second")
        print("=" * 70)
        
    except pymssql.OperationalError as e:
        error_code = e.args[0] if e.args else None
        if error_code == 40615:
            print("\n❌ FIREWALL ERROR - Cannot connect to database")
            print("\nPlease follow the instructions above to fix the firewall.")
        else:
            print(f"\n❌ Database Error: {e}")
            import traceback
            traceback.print_exc()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


Azure SQL Geolocation Translation Tool - Azure Translator API

📍 Getting your current IP address...
✓ Your current IP: 156.194.0.53
  (Use this IP if you need to add a firewall rule)

🔌 Connecting to Azure SQL Server...
   Server: almagraby.database.windows.net
   Database: Olist_dataset
✓ Connected successfully!

📋 Checking table structure...
✓ Table structure verified

📖 Reading untranslated cities from olist_geolocation_dataset_olist_geolocation_dataset...


C:\Users\youse\AppData\Local\Temp\ipykernel_23776\685796788.py:284: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ Found 7,960 unique cities to translate

📊 Estimated characters: 98,777
   Azure Free Tier: 2,000,000/month
   Percentage: 4.94%

🌐 Starting translation...
   Batch size: 100 cities/request
   Parallel workers: 5
Translating 7960 unique cities...
Translation error: 400 Client Error: Bad Request for url: https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&from=auto&to=en
Progress: 100/7960 cities (1%)
Translation error: 400 Client Error: Bad Request for url: https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&from=auto&to=en
Progress: 200/7960 cities (2%)
Translation error: 400 Client Error: Bad Request for url: https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&from=auto&to=en
Progress: 300/7960 cities (3%)
Translation error: 400 Client Error: Bad Request for url: https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&from=auto&to=enTranslation error: 400 Client Error: Bad Request for url: https://api.cognit

KeyboardInterrupt: 

In [ ]:
def translate_batch_azure(cities):
    """Translate a batch of cities using Azure Translator API with full error tracing"""
    if not cities:
        return {}
    
    valid_cities = [city for city in cities if pd.notna(city) and city != '']
    
    if not valid_cities:
        return {city: city for city in cities}
    
    path = '/translate'
    constructed_url = AZURE_TRANSLATOR_ENDPOINT.rstrip('/') + path  # ✅ Avoid double slashes
    
    params = {
        'api-version': '3.0',
        'from': 'auto',
        'to': 'en'
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    body = [{'text': str(city)} for city in valid_cities]
    translations = {}
    
    try:
        response = requests.post(constructed_url, params=params, headers=headers, json=body)
        
        # ✅ Trace the exact URL and request if the status is not 200
        if response.status_code != 200:
            print("\n🚨 TRANSLATION REQUEST FAILED 🚨")
            print("=" * 80)
            print(f"❌ Status Code: {response.status_code}")
            print(f"❌ URL: {response.url}")
            print(f"❌ Headers Sent: {headers}")
            print(f"❌ Parameters: {params}")
            print(f"❌ Body Sample (first 3 cities): {body[:3]}")
            try:
                print(f"❌ Response Text: {response.text[:500]}")  # limit for readability
            except:
                print("❌ Response Text: <unreadable>")
            print("=" * 80)
        
        response.raise_for_status()  # will raise HTTPError if not 200
        
        results = response.json()
        
        for city, result in zip(valid_cities, results):
            translated_text = result['translations'][0]['text']
            translations[city] = translated_text
        
        # Keep original values for NaN or empty
        for city in cities:
            if pd.isna(city) or city == '':
                translations[city] = city
        
    except requests.exceptions.RequestException as e:
        print(f"\nTranslation error: {e}")
        print("📎 Debug Info:")
        print(f"URL: {constructed_url}")
        print(f"Params: {params}")
        print(f"Body (first 3): {body[:3]}")
        print(f"Response (if any): {getattr(e.response, 'text', 'No response text')}")
        translations = {city: city for city in cities}
    
    except Exception as e:
        print(f"\n❌ Unexpected translation error: {e}")
        translations = {city: city for city in cities}
    
    return translations


In [ ]:
import pandas as pd
import pymssql
import requests
import uuid
import time
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed

# Suppress pandas warning about pymssql
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

# Azure SQL Server Configuration
SERVER = 'almagraby.database.windows.net'
DATABASE = 'Olist_dataset'
USERNAME = 'DEPI_Projecte'
PASSWORD = 'Almaghraby@240'
TABLE_NAME = 'olist_geolocation_dataset_olist_geolocation_dataset'  # Table containing geolocation_city column

# Azure Translator Configuration
AZURE_TRANSLATOR_KEY = 'AunHlUi5xLkxOBfcQnwIYmiLxh2PztKTD5b5Kbfu4uiLHt1alQKdJQQJ99BJACF24PCXJ3w3AAAbACOGZEWL'  # From Azure Portal
AZURE_TRANSLATOR_REGION = 'uaenorth'  # Your translator region (e.g., eastus, westeurope)
AZURE_TRANSLATOR_ENDPOINT = 'https://api.cognitive.microsofttranslator.com/'

# Performance Settings
# ⚠️ REDUCED BATCH_SIZE to avoid 400 errors with large batches
BATCH_SIZE = 50  # Reduced from 100 to handle problematic city names better
MAX_WORKERS = 5
DB_UPDATE_BATCH_SIZE = 50

def test_translator_connection():
    """Test the Azure Translator API connection with a simple request"""
    print("\n🧪 Testing Azure Translator API connection...")
    print("=" * 70)
    
    test_url = AZURE_TRANSLATOR_ENDPOINT + '/translate'
    print(f"Test URL: {test_url}")
    print(f"API Key: {'*' * 40}{AZURE_TRANSLATOR_KEY[-4:] if len(AZURE_TRANSLATOR_KEY) > 4 else '****'}")
    print(f"Region: {AZURE_TRANSLATOR_REGION}")
    
    params = {
        'api-version': '3.0',
        'to': 'en'
        # NOTE: No 'from' parameter - Azure auto-detects source language
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    body = [{'text': 'مرحبا'}]  # "Hello" in Arabic
    
    try:
        print("\nSending test request...")
        response = requests.post(test_url, params=params, headers=headers, json=body, timeout=10)
        
        print(f"Response Status: {response.status_code}")
        print(f"Response Headers: {dict(response.headers)}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ SUCCESS! Translation: {result[0]['translations'][0]['text']}")
            print("=" * 70)
            return True
        else:
            print(f"❌ FAILED! Status code: {response.status_code}")
            print(f"Response body: {response.text}")
            print("=" * 70)
            
            # Detailed error diagnosis
            if response.status_code == 401:
                print("\n🔴 ERROR 401: AUTHENTICATION FAILED")
                print("Possible causes:")
                print("1. Invalid API key (AZURE_TRANSLATOR_KEY)")
                print("2. API key regenerated in Azure Portal")
                print("\nTo fix:")
                print("• Go to Azure Portal → Your Translator Resource")
                print("• Keys and Endpoint → Copy Key 1 or Key 2")
                print("• Update AZURE_TRANSLATOR_KEY in your code")
            
            elif response.status_code == 403:
                print("\n🔴 ERROR 403: ACCESS DENIED")
                print("Possible causes:")
                print("1. Wrong region (AZURE_TRANSLATOR_REGION)")
                print("2. Resource not active or billing issue")
                print("\nTo fix:")
                print("• Go to Azure Portal → Your Translator Resource")
                print("• Check 'Location/Region' and update AZURE_TRANSLATOR_REGION")
                print("• Verify resource is active and has available quota")
            
            elif response.status_code == 404:
                print("\n🔴 ERROR 404: ENDPOINT NOT FOUND")
                print("Possible causes:")
                print("1. Wrong endpoint URL")
                print("2. Typo in the endpoint")
                print(f"\nCurrent endpoint: {AZURE_TRANSLATOR_ENDPOINT}")
                print("Should be: https://api.cognitive.microsofttranslator.com")
                print("\nTo fix:")
                print("• Verify AZURE_TRANSLATOR_ENDPOINT is exactly:")
                print("  'https://api.cognitive.microsofttranslator.com'")
                print("  (no trailing slash, no extra characters)")
            
            elif response.status_code == 429:
                print("\n🔴 ERROR 429: TOO MANY REQUESTS")
                print("You've exceeded your rate limit or quota")
                print("\nTo fix:")
                print("• Wait a few minutes and try again")
                print("• Check your quota in Azure Portal")
            
            print("=" * 70)
            return False
            
    except requests.exceptions.ConnectionError as e:
        print(f"❌ CONNECTION ERROR: {e}")
        print("\nPossible causes:")
        print("1. No internet connection")
        print("2. Firewall blocking outbound requests")
        print("3. DNS resolution issues")
        return False
    
    except requests.exceptions.Timeout as e:
        print(f"❌ TIMEOUT ERROR: {e}")
        print("\nThe request took too long. Check your internet connection.")
        return False
    
    except Exception as e:
        print(f"❌ UNEXPECTED ERROR: {e}")
        import traceback
        traceback.print_exc()
        return False

def get_connection():
    """Create a new database connection with retry logic"""
    max_retries = 3
    retry_delay = 5
    
    for attempt in range(max_retries):
        try:
            conn = pymssql.connect(
                server=SERVER,
                user=USERNAME,
                password=PASSWORD,
                database=DATABASE,
                port=1433,
                tds_version='7.4',
                timeout=60,
                login_timeout=60
            )
            return conn
        except pymssql.OperationalError as e:
            error_code = e.args[0] if e.args else None
            
            if error_code == 40615:
                print(f"\n⚠️  FIREWALL ERROR (Attempt {attempt + 1}/{max_retries})")
                print("=" * 70)
                print("Your Azure SQL Server firewall is blocking this connection.")
                print("\nTO FIX THIS:")
                print("1. Get your current IP:")
                
                try:
                    import requests
                    current_ip = requests.get('https://api.ipify.org', timeout=5).text
                    print(f"   Your IP: {current_ip}")
                except:
                    print("   Run: import requests; print(requests.get('https://api.ipify.org').text)")
                
                print("\n2. Add to Azure Firewall:")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Click '+ Add firewall rule'")
                print(f"   • Start IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print(f"   • End IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print("   • Click 'Save' and wait 2-5 minutes")
                print("\n3. OR Enable 'Allow Azure services' (Recommended):")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Toggle ON: 'Allow Azure services and resources to access this server'")
                print("   • Click 'Save'")
                print("=" * 70)
                
                if attempt < max_retries - 1:
                    print(f"\nWaiting {retry_delay} seconds before retry...")
                    time.sleep(retry_delay)
                else:
                    print("\n❌ Max retries reached. Please fix firewall and re-run.")
                    raise
            else:
                print(f"\n❌ Connection Error: {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    raise
        except Exception as e:
            print(f"\n❌ Unexpected Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                raise
    
    raise Exception("Failed to connect after all retries")

def translate_batch_azure(cities):
    """Translate a batch of cities using Azure Translator API"""
    if not cities:
        return {}
    
    valid_cities = [city for city in cities if pd.notna(city) and city != '']
    
    if not valid_cities:
        return {city: city for city in cities}
    
    # ════════════════════════════════════════════════════════════════
    # 🔧 FIX FOR 400 ERROR: Filter out problematic cities
    # ════════════════════════════════════════════════════════════════
    cleaned_cities = []
    city_map = {}  # Map cleaned -> original
    
    for city in valid_cities:
        city_str = str(city).strip()
        
        # Skip if too long (Azure limit is 10,000 characters per text element)
        if len(city_str) > 1000:
            print(f"⚠️  Skipping long city name ({len(city_str)} chars): {city_str[:50]}...")
            continue
        
        # Skip if empty after cleaning
        if not city_str:
            continue
        
        cleaned_cities.append(city_str)
        city_map[city_str] = city
    
    if not cleaned_cities:
        return {city: city for city in cities}
    
    # ════════════════════════════════════════════════════════════════
    
    path = '/translate'
    constructed_url = AZURE_TRANSLATOR_ENDPOINT + path
    
    # ════════════════════════════════════════════════════════════════
    # 🔧 FIX FOR 400 ERROR: Don't specify 'from' parameter
    # Azure auto-detects source language when 'from' is omitted
    # ════════════════════════════════════════════════════════════════
    params = {
        'api-version': '3.0',
        'to': 'en'
        # NOTE: 'from' parameter removed - Azure will auto-detect
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    # Build request body with cleaned cities
    body = [{'text': city} for city in cleaned_cities]
    translations = {}
    
    try:
        response = requests.post(constructed_url, params=params, headers=headers, json=body, timeout=30)
        response.raise_for_status()
        results = response.json()
        
        # Map translated results back to original city names
        for cleaned_city, result in zip(cleaned_cities, results):
            original_city = city_map[cleaned_city]
            translated_text = result['translations'][0]['text']
            translations[original_city] = translated_text
        
        # Handle empty/null cities
        for city in cities:
            if pd.isna(city) or city == '':
                translations[city] = city
            elif city not in translations:
                # City was filtered out, keep original
                translations[city] = city
        
    except requests.exceptions.HTTPError as e:
        # Enhanced error reporting for 400 errors
        if e.response.status_code == 400:
            print(f"\n❌ HTTP 400 Error: Bad Request")
            print(f"   Response: {e.response.text[:500]}")
            print(f"   Batch size: {len(cleaned_cities)} cities")
            
            # Try to identify the problematic city
            try:
                error_detail = e.response.json()
                print(f"   Error detail: {error_detail}")
            except:
                pass
            
            # Sample of cities in this batch
            print(f"   Sample cities in batch: {cleaned_cities[:3]}")
        else:
            print(f"\n❌ HTTP Error {e.response.status_code}: {e}")
            print(f"   URL: {constructed_url}")
            print(f"   Response: {e.response.text[:200]}")
        
        # Return original cities as fallback
        translations = {city: city for city in cities}
    except Exception as e:
        print(f"❌ Translation error: {e}")
        translations = {city: city for city in cities}
    
    return translations

def translate_all_cities(unique_cities, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    """Translate all unique cities using Azure Translator with parallel processing"""
    print(f"Translating {len(unique_cities)} unique cities...")
    
    city_batches = [unique_cities[i:i + batch_size] 
                    for i in range(0, len(unique_cities), batch_size)]
    
    all_translations = {}
    completed = 0
    failed_batches = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {executor.submit(translate_batch_azure, batch): batch 
                          for batch in city_batches}
        
        for future in as_completed(future_to_batch):
            try:
                translations = future.result()
                all_translations.update(translations)
                completed += len(translations)
                progress = (completed * 100) // len(unique_cities)
                print(f"Progress: {completed}/{len(unique_cities)} cities ({progress}%)")
            except Exception as e:
                failed_batches += 1
                print(f"Batch processing error: {e}")
    
    if failed_batches > 0:
        print(f"\n⚠️  Warning: {failed_batches} batches failed")
    
    return all_translations

def update_database_small_batches(translations, table_name, batch_size=DB_UPDATE_BATCH_SIZE):
    """Update database in SMALL batches to avoid query size limits"""
    if not translations:
        return
    
    valid_translations = {k: v for k, v in translations.items() 
                         if k and v and pd.notna(k) and pd.notna(v)}
    
    if not valid_translations:
        print("No valid translations to update")
        return
    
    total = len(valid_translations)
    print(f"Updating database with {total} translations in batches of {batch_size}...")
    
    items = list(valid_translations.items())
    updated_count = 0
    conn = None
    
    try:
        for i in range(0, len(items), batch_size):
            batch = items[i:i + batch_size]
            
            if conn:
                try:
                    conn.close()
                except:
                    pass
            
            conn = get_connection()
            cursor = conn.cursor()
            
            for original, translated in batch:
                try:
                    update_query = """
                    UPDATE {} 
                    SET geolocation_city_en = %s 
                    WHERE geolocation_city = %s 
                    AND (geolocation_city_en IS NULL OR geolocation_city_en = '')
                    """.format(table_name)
                    
                    cursor.execute(update_query, (translated, original))
                    updated_count += 1
                    
                except Exception as e:
                    print(f"Error updating '{original}': {e}")
                    continue
            
            conn.commit()
            cursor.close()
            
            progress = ((i + len(batch)) * 100) // total
            print(f"Database update progress: {updated_count}/{total} ({progress}%)")
    
    except Exception as e:
        print(f"Database update error: {e}")
    finally:
        if conn:
            try:
                conn.close()
            except:
                pass
    
    print(f"✓ Successfully updated {updated_count} records")

def main():
    try:
        start_time = time.time()
        
        print("=" * 70)
        print("Azure SQL Geolocation Translation Tool - Azure Translator API")
        print("=" * 70)
        
        # Check configuration
        print("\n🔍 Checking configuration...")
        
        if AZURE_TRANSLATOR_KEY == 'your_translator_key_here':
            print("❌ ERROR: Please update AZURE_TRANSLATOR_KEY")
            print("\nGet your key from:")
            print("Azure Portal → Translator Resource → Keys and Endpoint → Key 1")
            return
        
        if AZURE_TRANSLATOR_ENDPOINT != 'https://api.cognitive.microsofttranslator.com':
            print(f"⚠️  WARNING: Unusual endpoint detected")
            print(f"   Current: {AZURE_TRANSLATOR_ENDPOINT}")
            print(f"   Expected: https://api.cognitive.microsofttranslator.com")
        
        # TEST THE TRANSLATOR API FIRST
        if not test_translator_connection():
            print("\n❌ STOPPING: Translator API test failed")
            print("Please fix the configuration issues above and try again.")
            return
        
        # Display current IP for troubleshooting
        print("\n📍 Getting your current IP address...")
        try:
            current_ip = requests.get('https://api.ipify.org', timeout=5).text
            print(f"✓ Your current IP: {current_ip}")
        except:
            print("  (Could not detect IP)")
        
        print("\n🔌 Connecting to Azure SQL Server...")
        print(f"   Server: {SERVER}")
        print(f"   Database: {DATABASE}")
        
        conn = get_connection()
        cursor = conn.cursor()
        print("✓ Connected successfully!")
        
        # Check/create translation column
        print("\n📋 Checking table structure...")
        check_column_query = f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = '{TABLE_NAME}' 
            AND COLUMN_NAME = 'geolocation_city_en'
        )
        BEGIN
            ALTER TABLE {TABLE_NAME}
            ADD geolocation_city_en NVARCHAR(255)
        END
        """
        cursor.execute(check_column_query)
        conn.commit()
        print("✓ Table structure verified")
        
        # Read untranslated unique cities
        print(f"\n📖 Reading untranslated cities from {TABLE_NAME}...")
        query = f"""
        SELECT DISTINCT geolocation_city 
        FROM {TABLE_NAME} 
        WHERE (geolocation_city_en IS NULL OR geolocation_city_en = '')
        AND geolocation_city IS NOT NULL
        AND geolocation_city != ''
        """
        df = pd.read_sql(query, conn)
        
        cursor.close()
        conn.close()
        
        total_unique = len(df)
        print(f"✓ Found {total_unique:,} unique cities to translate")
        
        if total_unique == 0:
            print("\n✅ All cities are already translated!")
            return
        
        unique_cities = df['geolocation_city'].tolist()
        
        # Estimate characters
        total_chars = sum(len(str(city)) for city in unique_cities)
        print(f"\n📊 Estimated characters: {total_chars:,}")
        print(f"   Azure Free Tier: 2,000,000/month")
        print(f"   Percentage: {(total_chars/2000000)*100:.2f}%")
        
        if total_chars > 2000000:
            print("   ⚠️  WARNING: May exceed free tier limit")
        
        # Translate all unique cities
        print(f"\n🌐 Starting translation...")
        print(f"   Batch size: {BATCH_SIZE} cities/request")
        print(f"   Parallel workers: {MAX_WORKERS}")
        
        translations = translate_all_cities(unique_cities, BATCH_SIZE, MAX_WORKERS)
        
        # Check translation success
        successful_translations = sum(1 for orig, trans in translations.items() 
                                     if orig != trans and pd.notna(trans))
        print(f"\n✓ Successfully translated: {successful_translations}/{total_unique} cities")
        
        if successful_translations == 0:
            print("\n❌ ERROR: No cities were translated!")
            print("All translation requests failed. Check the error messages above.")
            return
        
        # Update database
        print(f"\n💾 Updating database...")
        print(f"   Update batch size: {DB_UPDATE_BATCH_SIZE} cities")
        
        update_database_small_batches(translations, TABLE_NAME, DB_UPDATE_BATCH_SIZE)
        
        # Show sample results
        print("\n📋 Sample translations:")
        conn = get_connection()
        sample_query = f"""
        SELECT TOP 10 geolocation_city, geolocation_city_en 
        FROM {TABLE_NAME} 
        WHERE geolocation_city_en IS NOT NULL
        ORDER BY geolocation_city
        """
        sample_df = pd.read_sql(sample_query, conn)
        print(sample_df.to_string(index=False))
        conn.close()
        
        # Show statistics
        elapsed_time = time.time() - start_time
        print(f"\n" + "=" * 70)
        print(f"✅ TRANSLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print(f"📊 Statistics:")
        print(f"   Unique cities found: {total_unique:,}")
        print(f"   Successfully translated: {successful_translations:,}")
        print(f"   Characters processed: {total_chars:,}")
        print(f"   Time taken: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
        if elapsed_time > 0:
            print(f"   Average speed: {total_unique/elapsed_time:.2f} cities/second")
        print("=" * 70)
        
    except pymssql.OperationalError as e:
        error_code = e.args[0] if e.args else None
        if error_code == 40615:
            print("\n❌ FIREWALL ERROR - Cannot connect to database")
            print("\nPlease follow the instructions above to fix the firewall.")
        else:
            print(f"\n❌ Database Error: {e}")
            import traceback
            traceback.print_exc()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Azure SQL Geolocation Translation Tool - Azure Translator API

🔍 Checking configuration...
⚠️  WARNING: Unusual endpoint detected
   Current: https://api.cognitive.microsofttranslator.com/
   Expected: https://api.cognitive.microsofttranslator.com

🧪 Testing Azure Translator API connection...
Test URL: https://api.cognitive.microsofttranslator.com//translate
API Key: ****************************************ZEWL
Region: uaenorth

Sending test request...
Response Status: 200
Response Headers: {'Date': 'Thu, 23 Oct 2025 16:17:53 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-expose-headers': 'X-RequestId,X-Metered-Usage,X-MT-System', 'x-requestid': '52b6a10e-365f-4c70-bf74-0697843a627c.EUWE.1023T1617', 'x-content-type-options': 'nosniff', 'x-metered-usage': '5', 'x-mt-system': 'Microsoft', 'x-envoy-upstream-service-time': '564', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'}
✅ SUCCES

C:\Users\youse\AppData\Local\Temp\ipykernel_23776\1399064206.py:478: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ Found 7,960 unique cities to translate

📊 Estimated characters: 98,777
   Azure Free Tier: 2,000,000/month
   Percentage: 4.94%

🌐 Starting translation...
   Batch size: 50 cities/request
   Parallel workers: 5
Translating 7960 unique cities...
Progress: 50/7960 cities (0%)
Progress: 100/7960 cities (1%)
Progress: 150/7960 cities (1%)
Progress: 200/7960 cities (2%)
Progress: 250/7960 cities (3%)
Progress: 300/7960 cities (3%)
Progress: 350/7960 cities (4%)
Progress: 400/7960 cities (5%)
Progress: 450/7960 cities (5%)
Progress: 500/7960 cities (6%)
Progress: 550/7960 cities (6%)
Progress: 600/7960 cities (7%)
Progress: 650/7960 cities (8%)
Progress: 700/7960 cities (8%)
Progress: 750/7960 cities (9%)
Progress: 800/7960 cities (10%)
Progress: 850/7960 cities (10%)
Progress: 900/7960 cities (11%)
Progress: 950/7960 cities (11%)
Progress: 1000/7960 cities (12%)
Progress: 1050/7960 cities (13%)
Progress: 1100/7960 cities (13%)
Progress: 1150/7960 cities (14%)
Progress: 1200/7960 cities (1

In [ ]:
pip install sqlalchemy pyodbc